In [74]:
import pandas as pd
import numpy as np
from keras.layers import Input, LSTM, Concatenate, Dense
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
# data preparation
data = pd.read_csv("/content/drive/My Drive/GOOG-year.csv")
#data =data[:50] # Juml row maksimal yang mau diambil sample 15 row

# memilih kolom open dan close saja
#data = data[['Open', 'Close','High','Low','Adj Close']] # all = Tanggal,Open,High,Low,Close,Adj Close,Volume
data = data[['Open']]

data.head()
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)
#print(data)

In [103]:
# membuat dataset dengan 90% data training dan 10% data testing
train_size = int(len(data) * 0.8) #226 total 252
train_data = data[:train_size] # 90% = 226 -252 = 26
test_data = data[train_size:]

# membuat dataset dengan input dan output
def create_dataset(data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data) - look_back): # print(len(data)) =252
        dataX.append(data[i:(i + look_back),:])
        dataY.append(data[i + look_back, :])
        
    #return np.array(dataX), np.array(dataY)
    return np.array(dataX), np.array(dataY)

# membuat dataset dengan input 2 hari sebelumnya dan output hari berikutnya
look_back = 2
trainX, trainY = create_dataset(train_data, look_back) #226,2
testX, testY = create_dataset(test_data, look_back)   #252,2
#print(trainX)

In [104]:
# membuat dataset dengan 90% data training dan 10% data testing
train_size = int(len(data) * 0.9) #226 total 252
train_data = data[:train_size] # 90% = 226 -252 = 26  /Train 90%
test_data = data[train_size:]                        #/ Test 10%

original_array=np.array(train_data) #trainX & trainY
train_data = original_array.ravel()   
#print(train_data)

original_array1=np.array(test_data) #testX & testY
test_data = original_array1.ravel()


#input data
open_data =train_data # data['Open'].values
close_data = train_data # data['Close'].values
high_data = train_data # data['High'].values
low_data = train_data # data['Low'].values
volume_data = train_data #  data['Volume'].values

#reshape data
open_data = open_data.reshape((len(open_data), 1,1))
close_data = close_data.reshape((len(close_data),1, 1))
high_data = high_data.reshape((len(high_data), 1,1))
low_data = low_data.reshape((len(low_data), 1,1))
volume_data = volume_data.reshape((len(volume_data),1, 1))
#print(open_data)

In [105]:
#input layer / Setting variable jadi 3 dimensi
open_in = Input(shape=(1,1))
close_in = Input(shape=(1,1))
high_in = Input(shape=(1,1))
low_in = Input(shape=(1,1))
volume_in = Input(shape=(1,1))

#LSTM layer
open_lstm = LSTM(50)(open_in)
close_lstm = LSTM(50)(close_in)
high_lstm = LSTM(50)(high_in)
low_lstm = LSTM(50)(low_in)
volume_lstm = LSTM(50)(volume_in)



In [106]:
#concatenate layer
merged = Concatenate()([open_lstm, close_lstm, high_lstm, low_lstm, volume_lstm])

#output layer
output = Dense(1)(merged)

#model
model = Model([open_in, close_in, high_in, low_in, volume_in], output)
model.compile(loss='mean_absolute_error', optimizer='adam')

#fit model
model.fit([open_data, close_data, high_data, low_data, volume_data], open_data, epochs=5, batch_size=32)
#model.fit([open_data, close_data, high_data, low_data, volume_data], data['Next_Close'], epochs=5, batch_size=32) # buatkan datanya

Epoch 1/5
8/8 [==============================] - 9s 7ms/step - loss: 0.4522
Epoch 2/5
8/8 [==============================] - 0s 6ms/step - loss: 0.3690
Epoch 3/5
8/8 [==============================] - 0s 10ms/step - loss: 0.2893
Epoch 4/5
8/8 [==============================] - 0s 6ms/step - loss: 0.2171
Epoch 5/5
8/8 [==============================] - 0s 6ms/step - loss: 0.1630


In [ ]:
#predict
predictions = model.predict([open_data, close_data, high_data, low_data, volume_data])

#evaluate
error = model.evaluate([open_data, close_data, high_data, low_data, volume_data], open_data)
#error = model.evaluate([open_data, close_data, high_data, low_data, volume_data], data['Close'])
print("Absolute Mean Error: ", error)


# inverting data kembali ke nilai asli
predictions = scaler.inverse_transform(predictions) # mungkin 2D
#train_data   = scaler.inverse_transform(train_data) # beda dimensi error

#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) #hapus
train_data=train_data.reshape(-1, 1)
train_data   = scaler.inverse_transform(train_data)

#test_data   = scaler.inverse_transform(test_data)
#testPredict = scaler.inverse_transform(testPredict)                       #Error
#testY = scaler.inverse_transform(testY)     

#plot
plt.plot(predictions, label='Predictions')
#plt.plot(data['Next_Close'], label='Actual')
#plt.plot(new_array1, label='Actual') # buatkan datanya
plt.plot(train_data, label='Training') # buatkan datanya
plt.legend()
plt.show()